 # 베이지안 머신러닝

###  박태영 교수
### 연세대학교 응용통계학과

### Outline

### 5. 나이브 베이즈 (2)
    5.3 범주형 나이브 베이즈
    5.4 다항 나이브 베이즈

#### 5.3 범주형 나이브 베이즈

- `K`개의 범주(`th`)가 존재
- 범주를 분류하기 위한 `N`개의 특성(`X`)이 각각 **범주형 변수**임
- 필요한 모듈을 설치

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

- 의학(피부과) 데이터 (출처 : https://archive.ics.uci.edu/ml/datasets/Dermatology)
- 비슷한 임상적 특징을 가지는 6개의 피부질환을 34개의 특성으로 분류하는 문제 
- 질병의 초기 단계에서는 다른 질병의 특징을 보일 수도 있어 진단의 어려움이 발생하므로 임상적 속성과 조직병리학적 속성을 모두 고려

- `th` 변수 (피부질환의 종류)
    - psoriasis(건선)
    - seboreic dermatitis(지루성 피부염) 
    - lichen planus(편평 태선)
    - pityriasis rosea(장미색 비강진) 
    - cronic dermatitis(만성 피부염)
    - pityriasis rubra pilaris(모공성 홍색 비강진)

- `X` 변수
    - **Clinical Attributes(임상적 속성):** (따로 지정되지 않은 경우, 0, 1, 2, 3의 값을 가짐)
        - 1: erythema
        - 2: scaling
        - 3: definite borders
        - 4: itching
        - 5: koebner phenomenon
        - 6: polygonal papules
        - 7: follicular papules
        - 8: oral mucosal involvement
        - 9: knee and elbow involvement
        - 10: scalp involvement
        - 11: family history, (0 or 1)
        - 34: Age (linear)

    - **Histopathological Attributes(조직병리학적 속성):** (0, 1, 2, 3의 값을 가짐)
        - 12: melanin incontinence
        - 13: eosinophils in the infiltrate
        - 14: PNL infiltrate
        - 15: fibrosis of the papillary dermis
        - 16: exocytosis
        - 17: acanthosis
        - 18: hyperkeratosis
        - 19: parakeratosis
        - 20: clubbing of the rete ridges
        - 21: elongation of the rete ridges
        - 22: thinning of the suprapapillary epidermis
        - 23: spongiform pustule
        - 24: munro microabcess
        - 25: focal hypergranulosis
        - 26: disappearance of the granular layer
        - 27: vacuolisation and damage of basal layer
        - 28: spongiosis
        - 29: saw-tooth appearance of retes
        - 30: follicular horn plug
        - 31: perifollicular parakeratosis
        - 32: inflammatory monoluclear inflitrate
        - 33: band-like infiltrate

In [2]:
data = pd.read_table('Dermatology.data',sep=',',header=None)

- 변수명을 일관되게 표현

In [3]:
data.columns = ['col'+str(i) for i in data.columns]

- `col33`에서 "`?`"의 결측 데이터를 가지는 8개 개체를 삭제

In [4]:
data = data[data.col33!='?']

- `col33`을 사분위수로 나누고 범주형 변수(`'0','1','2','3'`)로 변환

In [5]:
data['col33'] = pd.qcut(data['col33'].astype('int'),4,labels=[str(i) for i in range(0,4)])

- 특성 변수와 범주 변수를 정의

In [6]:
X  = data.drop('col34',axis=1) # 피부질환을 나타내는 변수 제외하고 특성 변수로 정의
th = data['col34'] # 피부질환을 나타내는 변수를 범주 변수로 정의
th.value_counts()

1    111
3     71
2     60
5     48
4     48
6     20
Name: col34, dtype: int64

- `X`의 범주에 대한 서수적 인코딩(ordinal encoding) 필요
- 순서는 중요하지 않으나, `0,2,3`을 `0,1,2`로 변환

In [7]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(X)
X = enc.transform(X)

- 데이터의 70%를 훈련용(`train`)으로 사용하고 나머지 30%를 테스트용(`test`)으로 사용하기 위해 분리
- `stratify=th`를 사용하여 훈련용 데이터와 테스트용 데이터의 `th` 비율이 동일하게 함 

In [8]:
X_train, X_test, th_train, th_test = train_test_split(X, th, test_size=0.3, random_state=1, stratify=th)

- 훈련용 데이터에 범주형 나이브 베이즈 학습

In [9]:
cnb = CategoricalNB()
cnb.fit(X_train, th_train)

CategoricalNB()

- 테스트용 데이터에 범주형 나이브 베이즈 성능 측정

In [10]:
th_pred = cnb.predict(X_test)
confusion_matrix(th_test, th_pred)

array([[34,  0,  0,  0,  0,  0],
       [ 0, 17,  0,  1,  0,  0],
       [ 0,  0, 21,  0,  0,  0],
       [ 0,  0,  0, 14,  0,  0],
       [ 0,  0,  0,  0, 15,  0],
       [ 0,  0,  0,  0,  0,  6]], dtype=int64)

In [11]:
print('Misclassified samples: %d' % (th_test != th_pred).sum())

Misclassified samples: 1


In [12]:
print('Accuracy: %.2f' % accuracy_score(th_test, th_pred))

Accuracy: 0.99


#### 5.4 다항 나이브 베이즈

- `K`개의 범주(`th`)가 존재
- 범주를 분류하기 위한 `N`개의 특성(`X`)이 각각 **가산형 변수**임 (0보다 크거나 같은 정수값을 가짐)

- 훈련용 데이터에 다항 나이브 베이즈 학습

In [13]:
mnb = MultinomialNB()
mnb.fit(X_train, th_train)

MultinomialNB()

- 테스트용 데이터에 다항 나이브 베이즈 성능 측정

In [14]:
th_pred = mnb.predict(X_test)
confusion_matrix(th_test, th_pred)

array([[34,  0,  0,  0,  0,  0],
       [ 0, 18,  0,  0,  0,  0],
       [ 0,  0, 21,  0,  0,  0],
       [ 0,  1,  0, 13,  0,  0],
       [ 0,  0,  0,  0, 15,  0],
       [ 0,  0,  0,  0,  0,  6]], dtype=int64)

In [15]:
print('Misclassified samples: %d' % (th_test != th_pred).sum())

Misclassified samples: 1


In [16]:
print('Accuracy: %.2f' % accuracy_score(th_test, th_pred))

Accuracy: 0.99
